## merge sumstats files

**THIS PIPELINE COULD BE CHANGED A LOT IN THE FUTURE WOR**

So far, the format of QTL data is a bit different for different people, including but not limited to whether there are ID columns, duplicate rows, snp names with ':' or '_', which cause a lot of problem in integration analysis. Hope this would be resolved in data-reproduction. But if not, or if there are some other differences... then you should change it at that time.

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/xqtl-pipeline/pipeline/mash_processing.ipynb merge_sumstats \
    --cwd /home/rf2872/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/

In [ ]:
[merge_sumstats_1]
parameter: cwd = path('./')
parameter: preffix = 'ROSMAP_Pseudo_eQTL_DLPFC_pQTL_MiGA'
parameter: suffix = 'norminal.cis_long_table.merged.vcf.gz'
parameter: gene_ref = '/mnt/vast/hpc/csg/molecular_phenotype_calling/reference_data/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list'
parameter: n = 22
n  = [x+1 for x in range(n)]
input: for_each = 'n'
output: f'{cwd:a}/prepare/{preffix}.chr{_n}.{suffix}'
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '40G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library(data.table)
        library(tidyverse)
        library(magrittr)
        gene_ref<-read.table("${gene_ref}")
        my_list <- list()
        datasets<-list.files(str_c('${_output:d}',"/data/"),full.names = T)
        for (i in 1:length(datasets)) {
            d_path <- datasets[[i]]
            if (basename(d_path) == "eqtl") {
                raw_sum <- data.table::fread(list.files(d_path, full.names = T) %>% .[grep(str_c("chr", '${_n}', ".norminal.cis_long_table"), .)])
                my_list[i] <- raw_sum
            } else {
                if (basename(d_path) == "DLPFC_pQTL") {
                    raw_sum <- data.table::fread(list.files(d_path, full.names = T) %>% .[grep(str_c("PEER.", '${_n}', ".norminal.cis_long_table"), .)])
                    new_sum <- data.table(ID = paste(stringr::str_split(raw_sum$molecular_trait_id, "_", simplify = T)[, 2], raw_sum$variant, sep = ":"), DLPFC_pQTL = raw_sum[, c(6:10, 12:13)] %>% apply(., 1, function(x) paste(x, collapse = ":")))
                }
                if (basename(d_path) %>% grep("MiGA", .) %>% length() > 0) {
                    raw_sum <- data.table::fread(list.files(d_path, full.names = T) %>% .[grep(str_c("chr", '${_n}', ".norminal.cis_long_table"), .)])
                    new_sum <- data.table(ID = paste(gene_ref$V5[match(raw_sum$molecular_trait_id, gene_ref$V4)], raw_sum$variant, sep = ":"), MiGA = raw_sum[, c(7, 8, 6, 3, 10, 4, 5)] %>% apply(., 1, function(x) paste(x, collapse = ":")))
                    colnames(new_sum)[2] %<>% basename(datasets[[3]])
                }
                my_list[i] <- new_sum[-which(duplicated(new_sum$ID)), ]
            }
        }
        merged_df <- Reduce(function(x, y) merge(x, y, by = "ID"), my_list)
        merged_df<-merged_df[,c('#CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT','ALL','Ast','End','Exc','Inh','Mic','OPC','Oli',basename( datasets)%>%.[.!="eqtl"])]
        fwrite(merged_df,'${_output}',sep = "\t")

In [ ]:
[merge_sumstats_2]
input: group = 1
output: f'{cwd:a}/input/{preffix}.chr{_n}.{suffix}'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = ncore, tags = f'{step_name}_{_output:bn}'
bash: expand = "$[ ]",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
    cd '$[_output:d]'
    zcat /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_celltypes_analysis/output/data_intergration/TensorQTL/ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.bed.processed_phenotype.per_chrom_ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.log2cpm.ALL_Ast_End_Exc_Inh_Mic_OPC_Oli.covariate.pca.resid.PEER.cov.chr$[_n].norminal.cis_long_table.merged.vcf.gz|head -n13 |gzip>head.$[_n].info.gz
    zcat head.${i}.info.gz $[_input] > $[_output]
    rm head.$[_n].info.gz

## Extract fine mapped signals

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_6/mash_processing.ipynb fine_mapped_signals_pQTL \
    --cwd /home/rf2872/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/  \
    --fine_mapped_file ~/Work/MR/2023.4_MR/data/fine-mapped/pQTL/pqtl.all_variants.tsv

In [ ]:
[fine_mapped_signals_pQTL]
parameter: cwd = path('./')
parameter: fine_mapped_file = path
parameter: pip = '0.5'
input: fine_mapped_file
output: f'{cwd:a}/SuSiE_output_pQTL_signals_table'
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library("stringr")
        #library(org.Hs.eg.db)
        library(tidyr)
        library(dplyr)
        library(purrr)
        library(tidyverse)
        cbind.fill <- function(...){
            nm <- list(...) 
            nm <- lapply(nm, as.matrix)
            n <- max(sapply(nm, nrow)) 
            do.call(cbind, lapply(nm, function (x) 
                rbind(x, matrix(, n-nrow(x), ncol(x))))) 
        }

        fm.pqtl<-data.table::fread("${_input}")

        final.snp<-c()
        final.snp.df<-NULL

            #rbind each cell file of each gene
            datalist<-fm.pqtl
            
            #1. for the snps outside of cs
            data.out<-datalist[datalist$"cs_order"==0,]
            data.out.pip<-data.out[data.out$pip>as.numeric(${pip}),]
            data.out.pip.snp<-data.out.pip$variants
            data.out.pip.snp.df<-data.frame(gene=data.out.pip$molecular_trait_id, 
                                            con=rep("outside_cs",length(data.out.pip.snp)),
                                            signals=data.out.pip$variants,
                                        cs_cellType=rep("pQTL",length(data.out.pip.snp)),
                                        cs_max_cellType=rep("pQTL",length(data.out.pip.snp)),
                                CSs=rep(NA,length(data.out.pip.snp)))
            data.out.pip.snp.df[1:5,]

            
            #2. for the snps in cs
            #reformat cs_table
            data.in<-datalist[datalist$"cs_order"!=0,]
            data.in$group<-paste0(data.in$molecular_trait_id,data.in$cs_order)

            cs.max.pip.snp.df<-NULL
            cs.max.pip.snp.df<-Reduce(function(...) rbind(...), lapply(unique(data.in$group),function(x){
            tmp<-data.in[data.in$group==x,]
            tmp.maxpip<-max(tmp$pip)
            tmp.max<-tmp[tmp$pip == tmp.maxpip,]
            tmp.max.pip<-data.frame(gene=rep(tmp.max$molecular_trait_id,nrow(tmp.max)), 
                                        con=rep("in_cs",nrow(tmp.max)),
                                        signals=tmp.max$variants,
                                        cs_cellType=rep("ALL",nrow(tmp.max)),
                                        cs_max_cellType=rep("pQTL",nrow(tmp.max)),
                                CSs=rep(tmp.max$cs_order,nrow(tmp.max)))
            return(tmp.max.pip)

        }))


            #final.snp<-c(data.out.pip.snp,cs.max.pip.snp)
            final.snp.df<-rbind(data.out.pip.snp.df,cs.max.pip.snp.df)
        final.snp.df$gene<-str_split(final.snp.df$gene,"_",simplify = T)[,2]
        write.table(final.snp.df,"${_output}",row.names = F)

## Extract fine mapped signals from multi QTL data


In [ ]:
[fine_mapped_signals_multiQTL]
parameter: cwd = path('./')
parameter: fine_mapped_file = path
parameter: pip = '0.5'
input: fine_mapped_file
output: f'{cwd:a}/SuSiE_output_pQTL_signals_table'
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
     